Reference CPU: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics        3.80 GHz

In [1]:
import pandas as pd
import torch
from joblib import cpu_count

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from sklearn.model_selection import train_test_split
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.preprocessing.pandas.pandas_preprocessor import PandasDataPreprocessor
from pyinstrument import Profiler
import pytorch_lightning as pl

c:\Users\1\ptls\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BATCH_SIZE = 48

In [3]:
profiler = Profiler()

def define_data():
    source_data = pd.read_csv(
        'https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true',
        compression='gzip')

    preprocessor = PandasDataPreprocessor(
        col_id='client_id',
        col_event_time='trans_date',
        event_time_transformation='none',
        cols_category=['small_group'],
        cols_numerical=['amount_rur'],
        return_records=True
    )
    return preprocessor, source_data


def define_model():
    trx_encoder_params = dict(embeddings_noise=0.003,
                              numeric_values={'amount_rur': 'identity'},
                              embeddings={'trans_date': {'in': 800, 'out': 16},
                                          'small_group': {'in': 250, 'out': 16}})

    seq_encoder = RnnSeqEncoder(trx_encoder=TrxEncoder(**trx_encoder_params), hidden_size=256, type='gru')
    model = CoLESModule(seq_encoder=seq_encoder, optimizer_partial=partial(torch.optim.Adam, lr=0.001),
                        lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9))
    return model

In [4]:
if __name__ == "__main__":
    accelerator = "cuda" if torch.cuda.is_available() else "cpu"
    device = 1 if torch.cuda.is_available() else "auto"
    data_loader_workers = 1 if torch.cuda.is_available() else cpu_count()
    preprocessor, source_data = define_data()
    model = define_model()

    dataset = preprocessor.fit_transform(source_data)
    train, test = train_test_split(dataset, test_size=0.2, random_state=42)
    
    len_filter = SeqLenFilter(min_seq_len=25)
    in_memory_dataset = MemoryMapDataset(data=train, i_filters=[len_filter])
    data_splitter = SampleSlices(split_count=5, cnt_min=25, cnt_max=200)
    coles_df = ColesDataset(data=in_memory_dataset, splitter=data_splitter)
    train_dl = PtlsDataModule(
        train_data=coles_df,
        train_num_workers=data_loader_workers,
        train_batch_size=BATCH_SIZE,
    )
    trainer = pl.Trainer(
        max_epochs=1,
        accelerator=accelerator,
        devices=device,
        enable_progress_bar=True,
        # prepare_data_per_node=False,
        # replace_sampler_ddp=False,
        # sync_batchnorm=True
    )

Creating Dask Server
Link Dask Server - http://192.168.0.105:8787/status


2024-12-05 16:36:24,177 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 3.15 GiB -- Worker memory limit: 3.79 GiB
2024-12-05 16:36:24,395 - distributed.worker.memory - WARNING - Worker is at 90% memory usage. Pausing worker.  Process memory: 3.42 GiB -- Worker memory limit: 3.79 GiB
2024-12-05 16:36:24,603 - distributed.worker.memory - WARNING - Worker is at 94% memory usage. Pausing worker.  Process memory: 3.60 GiB -- Worker memory limit: 3.79 GiB
2024-12-05 16:36:24,861 - distributed.worker.memory - WARNING - Worker is at 100% memory usage. Pausing worker.  Process memory: 3.80 GiB -- Worker memory limit: 3.79 GiB
2024-12-05 16:36:24,862 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.15 GiB

In [5]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

c:\Users\1\ptls\.venv\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 240 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.962     Total estimated model params size (MB)
17        Modules in train mode
0         Modules in eval mode
c:\Users\1\ptls\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initializat

logger.version = 145


2024-12-05 16:10:37,520 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-12-05 16:10:44,368 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-12-05 16:10:48,107 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-12-05 16:11:17,858 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-12-05 16:11:24,867 - distributed.utils_perf - WARNING - full garbage collections took 43% CPU time recently (threshold: 10%)
2024-12-05 16:11:41,816 - distributed.utils_perf - WARNING - full garbage collections took 43% CPU time recently (threshold: 10%)
2024-12-05 16:11:52,254 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-12-05 16:11:58,538 - distributed.utils_perf - WARNING - full garbage collections took

2024-12-05 16:12:16,348 - distributed.utils_perf - WARNING - full garbage collections took 46% CPU time recently (threshold: 10%)
2024-12-05 16:15:20,405 - distributed.utils_perf - WARNING - full garbage collections took 64% CPU time recently (threshold: 10%)


Epoch 0: 100%|██████████| 500/500 [06:34<00:00,  1.27it/s, v_num=145, seq_len=109.0]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 500/500 [06:34<00:00,  1.27it/s, v_num=145, seq_len=109.0]
{'loss': tensor(74.2729), 'seq_len': tensor(108.9583)}


### Save sequence encoder for other experiments

In [6]:
torch.save(model.state_dict(), "coles-emb.pt")

In [5]:
model.load_state_dict(torch.load("coles-emb.pt"))

C:\Users\1\AppData\Local\Temp\ipykernel_169628\282433253.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("coles-emb.pt"))


<All keys matched successfully>

## Inference 

In [6]:
from ptls.data_load.datasets import inference_data_loader
train_dl = inference_data_loader(train, num_workers=0, batch_size=BATCH_SIZE)
batch = next(iter(train_dl))
out = model(batch)
out

tensor([[-0.9735,  0.9915, -0.3622,  ...,  0.9922,  0.0333,  0.5107],
        [-0.9772,  0.9890, -0.4413,  ...,  0.9930,  0.3060,  0.3452],
        [-0.9811,  0.9920, -0.3003,  ...,  0.9889,  0.4440,  0.5506],
        ...,
        [-0.9769,  0.9884, -0.5546,  ...,  0.9910,  0.1383,  0.6291],
        [-0.9868,  0.9934, -0.4172,  ...,  0.9950,  0.2357,  0.6615],
        [-0.9772,  0.9937, -0.6985,  ...,  0.9937,  0.2834,  0.4918]],
       grad_fn=<SliceBackward0>)

In [7]:
from ptls.frames.inference_module import InferenceModule

inf_model = InferenceModule(model, model_out_name="coles-emb.pt", pandas_output=False)

batch = next(iter(train_dl))
out = inf_model(batch)
out

{'coles-emb.pt': tensor([[-0.9735,  0.9915, -0.3598,  ...,  0.9921,  0.0340,  0.5108],
         [-0.9772,  0.9890, -0.4384,  ...,  0.9930,  0.3072,  0.3461],
         [-0.9811,  0.9920, -0.2973,  ...,  0.9889,  0.4447,  0.5513],
         ...,
         [-0.9768,  0.9884, -0.5521,  ...,  0.9910,  0.1389,  0.6295],
         [-0.9868,  0.9934, -0.4147,  ...,  0.9950,  0.2362,  0.6621],
         [-0.9772,  0.9937, -0.6976,  ...,  0.9937,  0.2840,  0.4926]],
        grad_fn=<SliceBackward0>)}

## ONNX Inference 

In [8]:
from sklearn.ensemble import RandomForestClassifier
from ptls.frames.inference_module import ONNXInferenceModule
from ptls.data_load.datasets import inference_data_loader

%load_ext pyinstrument

torch.set_float32_matmul_precision('high')

train_dl = inference_data_loader(train, num_workers=0, batch_size=BATCH_SIZE, onnx=True)
test_dl = inference_data_loader(test, num_workers=0, batch_size=BATCH_SIZE, onnx=True)

onnx_model = ONNXInferenceModule(model, model_out_name="coles-emb.onnx", dl=train_dl).to(accelerator)
batch = next(iter(train_dl))
out = onnx_model(batch)
out

c:\Users\1\ptls\pytorch-lifestream\ptls\nn\seq_encoder\rnn_encoder.py:130: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.payload.size()[1] > 0, "Batch can'not have 0 transactions"
c:\Users\1\ptls\.venv\lib\site-packages\torch\onnx\symbolic_opset9.py:4545: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
c:\Users\1\ptls\.venv\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available provid

tensor([[-0.9736,  0.9917, -0.3616,  ...,  0.9922,  0.0328,  0.5117],
        [-0.9771,  0.9888, -0.4402,  ...,  0.9932,  0.3062,  0.3467],
        [-0.9810,  0.9922, -0.2993,  ...,  0.9888,  0.4436,  0.5518],
        ...,
        [-0.9771,  0.9883, -0.5537,  ...,  0.9912,  0.1379,  0.6299],
        [-0.9868,  0.9932, -0.4163,  ...,  0.9951,  0.2357,  0.6626],
        [-0.9771,  0.9937, -0.6982,  ...,  0.9937,  0.2837,  0.4929]],
       dtype=torch.float16)

### Inference with pl.trainer

In [ ]:
%%pyinstrument

train_embeds = torch.vstack(trainer.predict(onnx_model, train_dl, ))
test_embeds = torch.vstack(trainer.predict(onnx_model, test_dl))

In [12]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


In [13]:
embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5483333333333333

### Inference w/o pl.trainer (float32)

In [ ]:
%%pyinstrument

train_embeds = torch.vstack(onnx_model.predict(train_dl, dtype=torch.float32))
test_embeds = torch.vstack(onnx_model.predict(test_dl, dtype=torch.float32))

In [10]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


In [ ]:
%%pyinstrument

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

### Inference w/o pl.trainer (float16)

In [ ]:
%%pyinstrument

train_embeds = torch.vstack(onnx_model.predict(train_dl))
test_embeds = torch.vstack(onnx_model.predict(test_dl))

In [13]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


In [ ]:
%%pyinstrument

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

### Regular Inference

In [15]:
train_dl = inference_data_loader(train, num_workers=0, batch_size=BATCH_SIZE)
test_dl = inference_data_loader(test, num_workers=0, batch_size=BATCH_SIZE)

In [ ]:
%%pyinstrument

train_embeds = torch.vstack(trainer.predict(model, train_dl, ))
test_embeds = torch.vstack(trainer.predict(model, test_dl))

In [17]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


In [ ]:
%%pyinstrument

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)